## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.6.15"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.6.13

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_8wQ6jdtfOBltsICIvuotuK0p


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the upcoming weekend (Saturday and Sunday)
upcoming_saturday = today + timedelta((5-today.weekday()) % 7)
upcoming_sunday = upcoming_saturday + timedelta(days=1)

upcoming_saturday, upcoming_sunday
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 21, 2024", "count": 5, "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do in Atlanta on September 21, 2024", "askUserForLocation": true}, "webPages": {"webSearchUrl": "https:\/\/www.bing.com\/search?q=things+to+do+in+Atlanta+on+September+21%2c+2024", "totalEstimatedMatches": 46

In [4]:
Console.WriteLine(output.LastMessage)

Here are some events and activities happening in Atlanta on September 21 and 22, 2024:

### September 21, 2024
1. **[Atlanta Events September 2024](https://atlanta-ga.events/september/)**
   - Find various events like concerts, shows, sports, and more happening in Atlanta in September 2024.

2. **[Cool Things To Do in September in Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)**
   - Explore food festivals, haunted attractions, and music festivals. 

3. **[Atlanta Events - September 2024](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)**
   - Check out festivals, concerts, comedy shows, and more.

4. **[80+ Fall Festivals & Fairs Around Metro Atlanta](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)**
   - Enjoy carnival rides, music, arts, food, and more at various fall festivals.

5. **[20 Spectacular Things To Do This September In Atlanta](https://secretatlanta.co/things-to-do-september/)**
  

## Clean Up

In [5]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
